## Import the Segmented Corpus

In [ ]:
# If these packages are not installed:

# ! pip install git+https://github.com/iinemo/isanlp.git
# ! pip install isanlp_rst
# ! pip install hf_xet

In [ ]:
# === Import
# import pandas as pd
import sys
import json
from pathlib import Path
# from collections import Counter

import warnings
warnings.filterwarnings("ignore", message="`encoder_attention_mask` is deprecated")

# === Define the path to the auxiliary modules ===
ROOT = Path.cwd().parent
SRC = (ROOT / "src").resolve()

if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

# === import the module for rst work === <--- this is not ready yet
import importlib
import discourse.rst as rst

In [ ]:
# === Define the path to the data and the pattern for retrieval ==
HOME = Path.home()
DATA_DIR = (HOME / "My Drive" / "_VectorData" / "projects" / "identifying_depression_with_rst" / "data").resolve(strict=True)

corpus_path = DATA_DIR / "processed"
corpus_file = corpus_path / "segmented_corpora_320_token_split.json"

with open(corpus_file, "r") as file:
    corpora = json.load(file)

diagnoses_path = DATA_DIR / "interim"
diagnoses_file = diagnoses_path / "all_diagnoses.json"

with open(diagnoses_file, "r") as file:
    diagnoses = json.load(file)

In [ ]:
# === This is just in case for possible debugging (with verbose output) ===
# import transformers
# transformers.utils.logging.set_verbosity_info()
# transformers.utils.logging.enable_explicit_format()

## Visual Inspection

In [ ]:
# === A note on the structure of the resulting segmented corpora: ===
# Each separate corpus in the corpora is the value for the key indicating the name of this sepcific corpus (like 'ked' in this case)

corpora.keys()

In [ ]:
# === Further down, the tree the structure is as follows: ===
# The value of the key is a list of 2 items
# Where each item is also a list
# The first list is the original text either as a single list item if it has not been split
# Or as several itmes, which are the resulting chunks of the splitting pipeline upstream
# The second list is made up of the sentences returned by the sentence tokenizer as list items

# So, pulling the text (or the resulting chunks) for the "ked" coprus looks something like this:

corpora["ked"][0][0]

In [ ]:
len(corpora["ked"])

## Prep Everything for Running the RST Parser

In [ ]:
# In case we need to reload the module
rst = importlib.reload(rst)

In [ ]:
# === Initialize the Parser
# model = 'tchewik/isanlp_rst_v3'
# version = 'gumrrg'  # Choose from {'gumrrg', 'rstdt', 'rstreebank'}

rst.init_parser()

In [ ]:
# Put the target corpus as texts/chunks into a separate variable for easier navigation/iteration logic downstream

CORPUS_NAME = "ked"

corpus = [item[0] for item in corpora[CORPUS_NAME]] # grab only the texts/segments, not the texts as sentences

In [ ]:
# Double-check the strucutre of the corpus is what the parser would expect

corpus[:5]

In [ ]:
# Check that the nuber of segmented texts matches what we got in the previous stage (segmentation / 01_segemnt_documents.ipynb)

find_split_texts = [i for i in corpus if len(i) > 1]
len(find_split_texts)

## Parsing the Selected Corpus

In [ ]:
# === This works on the specific corpus from the corpora: ===

# 
# The code expects the corpus to be a list of items, where each item is also a list
# containing the whole document as one list item or the doucment in two or more chunks (as list items)
# e.g. [['doc_1'], ['seg1_of_doc_2', 'seg2_of_doc_2'], ['doc_3'] ... ]
# There is also a safeguard to normalize each item in corpus to a list[str]
# for cases where the structure of the corpus may be like
# ["doc_1", ["seg1_of_doc_2", "seg2_of_doc_2"], "doc_3", ... ]

parsed_corpus = rst.parse_corpus(corpus)

## A Bit of Visual Inspection

In [ ]:
# === See if any errors have been logged
parsed_corpus[1]

In [ ]:
# Get rid of the errors log and keep the parsed corpus only
parsed_corpus = parsed_corpus[0]

# Check the number of items in the corpus
# Shoud match the number of documents (segmented or otherwise) in the initial corpus
len(parsed_corpus)

In [ ]:
# === What does the parser return and how is it structured in our 'segmented' corpus?

# First, each item is a list of at least one item -- a dictionary returned by the parser:


In [ ]:
# Further down the tree, each such dictionary has the key "rst", which stores the results of parsing as a list of one item (in our case)
# This item is the RST object/tree proper
# It can be explored using the 'vars' function:

parsed_corpus[0]

## Extract the Features

In [ ]:
# Run the main function to extract all the RST features

rst_res = rst.extract_all_rst_features(parsed_corpus)

In [ ]:
# The function returns a tuple where the first item is the list where each item is a dict of RST features (one for each text)

all_features = rst_res[0]
len(all_features)

In [ ]:
# The second item in the tuple is a list (a set) of all the relations that the parser identified in the coprus
all_realtions = rst_res[1]
len(all_realtions)

In [ ]:
# Visual inspection
all_realtions

In [ ]:
all_features[:2]

## Follow-up (in the Notebook for Now, in the Module Later)

### (Explicitly) Transform the Diagnosis into Labels

In [ ]:
# The diagnonses database is structured similar to the corpora
# It's a dictionary with keys for the names of corpora and the diagnoses are lists of string items

# Interate over the database and get only unique labels used
labels = {}
for k, v in diagnoses.items():
    labels.setdefault(k, []).extend(set(v))

In [ ]:
labels

In [ ]:
# Set up the mapping for encoding the labels
# 1 for positive diagnonses (like anxiety, depression), 0 for negative

map_ = {'высокая депрессивность': 1,
        'нет депрессивности': 0,
        'низкая депрессивность': 0,
        'здоровые': 0,
        'депрессия': 1,
        'высокая тревожность': 1,
        'нет тревожности': 0,
        'низкая тревожность': 0}

In [ ]:
# Make a y set with all the encoded labales for the corpora

y_all = {}

for k, v in diagnoses.items():
    y_all[k] = [map_.get(s, None) for s in v]

In [ ]:
# Double check the mapping is correct

y_all["kldd"]

In [ ]:
# Must be int

type(y_all["kldd"][0])

In [ ]:
# Double check the mapping is correct

diagnoses["kldd"]

## Visaul Inspection of the Most Obvious Differences in Terms of Relation Counts and Proportions

In [ ]:
# A helper to count all the instances of a relation in a sub-corpus
from collections import Counter

def count_relations(sub_coprus_features):
    relation_counter = Counter()
    for item in sub_coprus_features:
        relation_counter.update(item["relation_counts"])

    return relation_counter.most_common()

def convert_abs_rel_counts(relation_counts):
    total = 0
    
    relative_rels = []
    for item in relation_counts:
        total += int(item[1])
    
    for item in relation_counts:
        interim_rels = (item[0], item[1]/total)
        relative_rels.append(interim_rels)
    return relative_rels

In [ ]:
all_features_pos = []
all_features_neg = []

for idx, val in enumerate(y_all[CORPUS_NAME]):
    if val == 0:
        all_features_neg.append(all_features[idx])
    elif val == 1:
        all_features_pos.append(all_features[idx])
    else:
        print("Something is wrong, the document/features did not match any label")

In [ ]:
relation_counts_pos = count_relations(all_features_pos)
relation_counts_neg = count_relations(all_features_neg)

relative_relation_counts_pos = convert_abs_rel_counts(relation_counts_pos)
relative_relation_counts_neg = convert_abs_rel_counts(relation_counts_neg)

In [ ]:
relation_counts_pos

In [ ]:
relation_counts_neg

In [ ]:
relative_relation_counts_pos

In [ ]:
relative_relation_counts_neg

In [ ]:
causal_rel_props_pos = [item['relation_proportions'].get("causal", 0.0) for item in all_features_pos]
causal_rel_props_neg = [item['relation_proportions'].get("causal", 0.0) for item in all_features_neg]

## Investigate Why A Segment is Reported to be over 512

In [ ]:
import logging, transformers
transformers.utils.logging.set_verbosity_info()
transformers.utils.logging.enable_explicit_format()

parsed_corpus, offenders = [], []
for di, doc in enumerate(corpus):
    for si, seg in enumerate(_as_segments(doc)):
        print(f"processing doc:{di}, seg:{si}", flush=True)
        try:
            _ = parser(seg)                 # just probe once
            parsed_corpus.append(None)      # we're not keeping results here
        except Exception as e:
            offenders.append((di, si, str(e)))


In [ ]:
from transformers import AutoTokenizer

# model_name = "DeepPavlov/rubert-base-cased"
# model_name = "ai-forever/ruBert-base"
model_name = "sberbank-ai/ruBert-large"


tok = AutoTokenizer.from_pretrained(model_name)
for item in corpus:
    for segment in item:
        print(len(tok(normalize_for_parser(segment), add_special_tokens=True)["input_ids"]))

In [ ]:
res_temp = parser(normalize_for_parser(corpus[106][2]))

In [ ]:
import re, unicodedata
from transformers import AutoTokenizer

# 1) close-enough tokenizer to what the RST parser likely uses
xlmr = AutoTokenizer.from_pretrained("xlm-roberta-base")

def single_len(text: str) -> int:
    return len(xlmr(text, add_special_tokens=True)["input_ids"])

def pair_len(a: str, b: str) -> int:
    # this mimics how RoBERTa encodes pairs: <s>a</s></s>b</s>
    return len(xlmr(a, b, add_special_tokens=True)["input_ids"])

def max_pair_len(text: str) -> int:
    # super-naive sentence split is fine for triage; plug in your splitter if you want
    sents = [t for t in re.split(r'(?<=[.!?])\s+', text) if t.strip()]
    m = 0
    for i in range(len(sents)-1):
        m = max(m, pair_len(sents[i], sents[i+1]))
        if m >= 512:  # early exit
            break
    return m

offenders = []
for di, doc in enumerate(corpus):
    for si, raw in enumerate(_as_segments(doc)):
        seg = normalize_for_parser(raw)
        s_len = single_len(seg)
        p_len = max_pair_len(seg)
        if s_len >= 512 or p_len >= 512 or p_len >= 508:   # 508≈practical tripwire
            offenders.append((di, si, s_len, p_len))

print(f"Found {len(offenders)} risky segments (doc_idx, seg_idx, single_len, max_pair_len):")
for row in offenders[:20]:
    print(row)

## NEXT:
* Walk the RST Trees and Extract the Data (this part is WIP)